# 🏗️ Normalização de Endereços - Bronze → Prata

Este notebook:
1. Carrega endereços da camada Bronze
2. Normaliza cada endereço para o padrão brasileiro (ABNT NBR 14725 / Correios)
3. Extrai e estrutura componentes do endereço
4. Salva dados normalizados na camada Prata

## Formato Padrão Brasileiro:
```
[Tipo Logradouro] [Nome Logradouro], [Número] - [Complemento] - [Bairro] - [Município]/[UF] - CEP [CEP]
```

## 1. Configuração e Importação

In [ ]:
import pandas as pd
import numpy as np
from minio import Minio
import io
from datetime import datetime
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Adicionar diretório atual ao path para importar o normalizador
sys.path.insert(0, os.getcwd())

# Importar normalizador
from normalizar_enderecos_brasileiros import NormalizadorEndereco

# Configurações MinIO
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

print("✅ Bibliotecas importadas com sucesso!")
print("✅ Normalizador de endereços carregado!")

## 2. Conexão com MinIO

In [ ]:
# Conectar ao MinIO
minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

print("✅ Conectado ao MinIO")

## 3. Carregar Dados da Camada Bronze

In [ ]:
# Listar arquivos disponíveis na camada Bronze
print("📁 Buscando arquivos na camada Bronze - Cidadãos...")
print("=" * 80)

objects = list(minio_client.list_objects(BUCKET_NAME, prefix="bronze/simulado/cidadaos/", recursive=True))

if not objects:
    print("⚠️  Nenhum arquivo encontrado na camada Bronze.")
    print("   Execute primeiro o script de geração de dados.")
else:
    print(f"✅ {len(objects)} arquivo(s) encontrado(s)")
    for obj in objects:
        tamanho_mb = obj.size / 1024 / 1024
        print(f"  📄 {obj.object_name} ({tamanho_mb:.2f} MB)")

In [ ]:
# Carregar dados da camada Bronze
if objects:
    # Pegar o arquivo mais recente
    arquivo_mais_recente = max(objects, key=lambda x: x.last_modified)
    print(f"\n📂 Carregando: {arquivo_mais_recente.object_name}")
    
    # Ler dados
    response = minio_client.get_object(BUCKET_NAME, arquivo_mais_recente.object_name)
    df_bronze = pd.read_parquet(io.BytesIO(response.read()))
    
    print(f"✅ Dados carregados: {len(df_bronze):,} registros")
    print(f"\n📊 Estrutura dos dados:")
    print(f"  • Colunas: {list(df_bronze.columns)}")
    print(f"  • Shape: {df_bronze.shape}")
    
    # Mostrar amostra
    print(f"\n📝 Amostra dos dados (primeiras 3 linhas):")
    display(df_bronze.head(3))
else:
    df_bronze = None
    print("⚠️  Nenhum dado para processar.")

## 4. Inicializar Normalizador

In [ ]:
# Inicializar normalizador
normalizador = NormalizadorEndereco()

print("✅ Normalizador inicializado")
print("\n🧪 Testando normalizador com exemplo:")

if df_bronze is not None and len(df_bronze) > 0:
    exemplo_endereco = df_bronze['endereco'].iloc[0]
    print(f"\n📝 Endereço original:")
    print(f"   {exemplo_endereco}")
    
    componentes = normalizador.normalizar(exemplo_endereco)
    print(f"\n🔍 Componentes extraídos:")
    for chave, valor in componentes.items():
        if valor:
            print(f"   • {chave}: {valor}")
    
    endereco_normalizado = normalizador.normalizar_completo(exemplo_endereco)
    print(f"\n✅ Endereço normalizado:")
    print(f"   {endereco_normalizado}")

## 5. Normalizar Todos os Endereços

In [ ]:
if df_bronze is not None:
    print("🔄 Normalizando endereços...")
    print("=" * 80)
    
    # Criar DataFrame para armazenar dados normalizados
    dados_normalizados = []
    
    total_registros = len(df_bronze)
    
    for idx, row in df_bronze.iterrows():
        # Normalizar endereço
        componentes = normalizador.normalizar(row['endereco'])
        endereco_normalizado = normalizador.normalizar_completo(row['endereco'])
        
        # Criar registro normalizado
        registro = {
            # Dados do cidadão (mantidos da bronze)
            'cpf': row['cpf'],
            'nome': row['nome'],
            'telefone': row['telefone'],
            'tipo_telefone': row['tipo_telefone'],
            'email': row['email'],
            'numero_endereco': row['numero_endereco'],
            'total_enderecos': row['total_enderecos'],
            
            # Endereço original (para referência)
            'endereco_original': row['endereco'],
            
            # Endereço normalizado (formato padrão)
            'endereco_normalizado': endereco_normalizado,
            
            # Componentes estruturados
            'tipo_logradouro': componentes.get('tipo_logradouro'),
            'nome_logradouro': componentes.get('nome_logradouro'),
            'numero_imovel': componentes.get('numero'),
            'complemento': componentes.get('complemento'),
            'bairro': componentes.get('bairro'),
            'municipio': componentes.get('municipio'),
            'uf': componentes.get('uf'),
            'cep': componentes.get('cep'),
            
            # Flags de qualidade
            'tem_complemento': componentes.get('complemento') is not None,
            'tem_bairro': componentes.get('bairro') is not None,
            'tem_municipio': componentes.get('municipio') is not None,
            'tem_uf': componentes.get('uf') is not None,
            'tem_cep': componentes.get('cep') is not None,
            'completo': all([
                componentes.get('tipo_logradouro'),
                componentes.get('nome_logradouro'),
                componentes.get('numero'),
                componentes.get('bairro'),
                componentes.get('municipio'),
                componentes.get('uf'),
                componentes.get('cep')
            ])
        }
        
        dados_normalizados.append(registro)
        
        # Progresso
        if (idx + 1) % 10000 == 0:
            progresso = ((idx + 1) / total_registros) * 100
            print(f"  Progresso: {idx + 1:,}/{total_registros:,} ({progresso:.1f}%)")
    
    # Criar DataFrame normalizado
    df_prata = pd.DataFrame(dados_normalizados)
    
    print(f"\n✅ Normalização concluída!")
    print(f"   • Total de registros processados: {len(df_prata):,}")
    print(f"   • Registros com endereço completo: {df_prata['completo'].sum():,} ({df_prata['completo'].sum()/len(df_prata)*100:.1f}%)")
    
    print(f"\n📊 Estatísticas de qualidade:")
    print(f"   • Com complemento: {df_prata['tem_complemento'].sum():,} ({df_prata['tem_complemento'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com bairro: {df_prata['tem_bairro'].sum():,} ({df_prata['tem_bairro'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com município: {df_prata['tem_municipio'].sum():,} ({df_prata['tem_municipio'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com UF: {df_prata['tem_uf'].sum():,} ({df_prata['tem_uf'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com CEP: {df_prata['tem_cep'].sum():,} ({df_prata['tem_cep'].sum()/len(df_prata)*100:.1f}%)")
else:
    df_prata = None
    print("⚠️  Nenhum dado para normalizar.")

## 6. Visualizar Dados Normalizados

In [ ]:
if df_prata is not None:
    print("📊 Amostra dos dados normalizados:")
    print("=" * 80)
    
    # Mostrar algumas colunas principais
    colunas_principais = [
        'cpf', 'nome', 'endereco_original', 'endereco_normalizado',
        'tipo_logradouro', 'nome_logradouro', 'numero_imovel',
        'complemento', 'bairro', 'municipio', 'uf', 'cep', 'completo'
    ]
    
    display(df_prata[colunas_principais].head(10))
    
    print(f"\n📋 Estrutura completa do DataFrame:")
    print(f"   • Total de colunas: {len(df_prata.columns)}")
    print(f"   • Colunas: {list(df_prata.columns)}")

In [ ]:
if df_prata is not None:
    print("🔍 Exemplos de normalização:")
    print("=" * 80)
    
    # Mostrar alguns exemplos lado a lado
    exemplos = df_prata.head(5)
    
    for idx, row in exemplos.iterrows():
        print(f"\n📝 Exemplo {idx + 1}:")
        print(f"   CPF: {row['cpf']}")
        print(f"   Nome: {row['nome']}")
        print(f"   \n   ❌ Original:")
        print(f"      {row['endereco_original']}")
        print(f"   \n   ✅ Normalizado:")
        print(f"      {row['endereco_normalizado']}")
        print(f"   \n   📋 Componentes:")
        if row['tipo_logradouro']:
            print(f"      • Tipo: {row['tipo_logradouro']}")
        if row['nome_logradouro']:
            print(f"      • Logradouro: {row['nome_logradouro']}")
        if row['numero_imovel']:
            print(f"      • Número: {row['numero_imovel']}")
        if row['complemento']:
            print(f"      • Complemento: {row['complemento']}")
        if row['bairro']:
            print(f"      • Bairro: {row['bairro']}")
        if row['municipio']:
            print(f"      • Município: {row['municipio']}")
        if row['uf']:
            print(f"      • UF: {row['uf']}")
        if row['cep']:
            print(f"      • CEP: {row['cep']}")
        print(f"      • Completo: {'✅' if row['completo'] else '❌'}")
        print("-" * 80)

## 7. Análise de Qualidade dos Dados Normalizados

In [ ]:
if df_prata is not None:
    print("📊 Análise de Qualidade dos Dados Normalizados")
    print("=" * 80)
    
    # Distribuição por UF
    print(f"\n🗺️ Distribuição por Estado (Top 10):")
    uf_dist = df_prata['uf'].value_counts().head(10)
    for uf, qtd in uf_dist.items():
        if pd.notna(uf):
            percentual = (qtd / len(df_prata)) * 100
            print(f"   • {uf}: {qtd:,} ({percentual:.2f}%)")
    
    # Distribuição por município
    print(f"\n🏙️ Distribuição por Município (Top 10):")
    municipio_dist = df_prata['municipio'].value_counts().head(10)
    for municipio, qtd in municipio_dist.items():
        if pd.notna(municipio):
            percentual = (qtd / len(df_prata)) * 100
            print(f"   • {municipio}: {qtd:,} ({percentual:.2f}%)")
    
    # Tipos de logradouro
    print(f"\n🛣️ Tipos de Logradouro (Top 10):")
    tipo_dist = df_prata['tipo_logradouro'].value_counts().head(10)
    for tipo, qtd in tipo_dist.items():
        if pd.notna(tipo):
            percentual = (qtd / len(df_prata)) * 100
            print(f"   • {tipo}: {qtd:,} ({percentual:.2f}%)")
    
    # Estatísticas de completude
    print(f"\n✅ Completude dos Endereços:")
    print(f"   • Endereços completos: {df_prata['completo'].sum():,} ({df_prata['completo'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Endereços incompletos: {(~df_prata['completo']).sum():,} ({(~df_prata['completo']).sum()/len(df_prata)*100:.1f}%)")

## 8. Salvar na Camada Prata

In [ ]:
def save_to_prata(df, dataset_name, partition_date=None):
    """Salva DataFrame na camada Prata em formato Parquet"""
    if partition_date is None:
        partition_date = datetime.now().strftime('%Y%m%d')

    object_name = f"prata/{dataset_name}/dt={partition_date}/data.parquet"

    try:
        # Converter para Parquet
        buffer = io.BytesIO()
        df.to_parquet(buffer, index=False, engine='pyarrow', compression='snappy')
        buffer.seek(0)

        # Upload para MinIO
        minio_client.put_object(
            BUCKET_NAME,
            object_name,
            buffer,
            length=buffer.getbuffer().nbytes,
            content_type='application/octet-stream'
        )

        tamanho_mb = buffer.getbuffer().nbytes / 1024 / 1024
        print(f"✅ Prata: {object_name}")
        print(f"   • {len(df):,} registros")
        print(f"   • {tamanho_mb:.2f} MB")
        return True
    except Exception as e:
        print(f"❌ Erro ao salvar {object_name}: {e}")
        return False

print("💾 Função de salvamento definida!")

In [ ]:
if df_prata is not None:
    print("💾 Salvando dados normalizados na camada Prata...")
    print("=" * 80)
    
    # Salvar na camada Prata
    sucesso = save_to_prata(df_prata, 'cidadaos_enderecos_normalizados')
    
    if sucesso:
        print(f"\n✅ Dados salvos com sucesso na camada Prata!")
        print(f"\n📁 Localização: prata/cidadaos_enderecos_normalizados/dt=YYYYMMDD/data.parquet")
        
        # Verificar arquivo salvo
        print(f"\n🔍 Verificando arquivo salvo...")
        partition_date = datetime.now().strftime('%Y%m%d')
        object_name = f"prata/cidadaos_enderecos_normalizados/dt={partition_date}/data.parquet"
        
        try:
            response = minio_client.stat_object(BUCKET_NAME, object_name)
            tamanho_mb = response.size / 1024 / 1024
            print(f"   ✅ Arquivo confirmado: {tamanho_mb:.2f} MB")
        except Exception as e:
            print(f"   ⚠️  Erro ao verificar arquivo: {e}")
    else:
        print(f"\n❌ Erro ao salvar dados.")
else:
    print("⚠️  Nenhum dado para salvar.")

## 9. Resumo Final

In [ ]:
if df_prata is not None:
    print("=" * 80)
    print("📋 RESUMO DA NORMALIZAÇÃO")
    print("=" * 80)
    
    print(f"\n📊 Estatísticas Gerais:")
    print(f"   • Total de registros processados: {len(df_prata):,}")
    print(f"   • Cidadãos únicos: {df_prata['cpf'].nunique():,}")
    print(f"   • Endereços únicos normalizados: {df_prata['endereco_normalizado'].nunique():,}")
    
    print(f"\n✅ Qualidade dos Dados:")
    print(f"   • Endereços completos: {df_prata['completo'].sum():,} ({df_prata['completo'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com CEP: {df_prata['tem_cep'].sum():,} ({df_prata['tem_cep'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com UF: {df_prata['tem_uf'].sum():,} ({df_prata['tem_uf'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com município: {df_prata['tem_municipio'].sum():,} ({df_prata['tem_municipio'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com bairro: {df_prata['tem_bairro'].sum():,} ({df_prata['tem_bairro'].sum()/len(df_prata)*100:.1f}%)")
    print(f"   • Com complemento: {df_prata['tem_complemento'].sum():,} ({df_prata['tem_complemento'].sum()/len(df_prata)*100:.1f}%)")
    
    print(f"\n🗺️ Distribuição Geográfica:")
    print(f"   • Estados únicos: {df_prata['uf'].nunique()}")
    print(f"   • Municípios únicos: {df_prata['municipio'].nunique()}")
    
    print(f"\n📁 Dados Salvos:")
    partition_date = datetime.now().strftime('%Y%m%d')
    print(f"   • Camada: Prata")
    print(f"   • Dataset: cidadaos_enderecos_normalizados")
    print(f"   • Partição: dt={partition_date}")
    
    print(f"\n✅ Processo concluído com sucesso!")